In [5]:
import os
import cv2
import sys
import dlib
import numpy as np
import pickle as pkl
from skimage import io
from sklearn.decomposition import PCA

## Extract keypoint

In [18]:
predictor_path = "F:/code/avatar/Pre-trained_model/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
def get_facial_landmarks(filename):
    image = io.imread(filename);
    # detect face(s)
    dets = detector(image, 1);
    shape = np.empty([1,1])
    for k, d in enumerate(dets):
        # Get the landmarks/parts for the face in box d.
        shape = predictor(image, d);
        shape = shape_to_np(shape);

    return shape

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)
    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    # return the list of (x, y)-coordinates
    return coords

def getTilt(keypoints_mn):
    # Remove in plane rotation using the eyes
    eyes_kp = np.array(keypoints_mn[36:47])
    x = eyes_kp[:, 0]
    y = -1*eyes_kp[:, 1]
    # print('X:', x)
    # print('Y:', y)
    m = np.polyfit(x, y, 1)
    tilt = np.degrees(np.arctan(m[0]))
    return tilt

def getKeypointFeatures(keypoints):
    # Mean Normalize the keypoints wrt the center of the mouth
    # Leads to face position invariancy
    mouth_kp_mean = np.average(keypoints[48:67], 0)
    keypoints_mn = keypoints - mouth_kp_mean

    # Remove tilt
    x_dash = keypoints_mn[:, 0]
    y_dash = keypoints_mn[:, 1]
    theta = np.deg2rad(getTilt(keypoints_mn))
    c = np.cos(theta);	s = np.sin(theta)
    x = x_dash*c - y_dash*s	# x = x'cos(theta)-y'sin(theta)
    y = x_dash*s + y_dash*c # y = x'sin(theta)+y'cos(theta)
    keypoints_tilt = np.hstack((x.reshape((-1,1)), y.reshape((-1,1))))

    # Normalize
    N = np.linalg.norm(keypoints_tilt, 2)
    return [keypoints_tilt/N, N, theta, mouth_kp_mean]

In [30]:
image_root = 'F:/dataset/Avatar/Obama/clip/image_crop/00001_1'
names = os.listdir(image_root)
keypoint_list = []
for name in sorted(names):
    image_path = os.path.join(image_root,name)
    keypoint = get_facial_landmarks(image_path)
    # 旋转 并 利用L2进行normalization
    l = getKeypointFeatures(keypoint)
    unit_kp, N, tilt, mean = l[0], l[1], l[2], l[3]
    kp_mouth = unit_kp[48:68]
    # [处理过的嘴部特征，L2范数，旋转角度，嘴部特征平局值，处理过的所有特征，处理前的特征]
    store_list = [kp_mouth, N, tilt, mean, unit_kp, keypoint]
    keypoint_list.append(store_list)

In [35]:
store_list[0].shape, store_list[1], store_list[2], store_list[3].shape, store_list[4].shape, store_list[5].shape

((20, 2), 291.66931916808994, -0.04399697319950468, (2,), (68, 2), (68, 2))

In [69]:
kp_mouth.shape

(20, 2)

## PCA mouth keypoint

In [70]:
new_list =[]
for kp in keypoint_list:
    kp_mouth = kp[0]
    x = kp_mouth[:, 0].reshape((1, -1))
    y = kp_mouth[:, 1].reshape((1, -1))
    X = np.hstack((x, y)).reshape((-1)).tolist()
    new_list.append(X)

In [75]:
X = np.array(new_list)

In [76]:
X.shape

(108, 40)

In [77]:
pca = PCA(n_components=8)
pca.fit(X)

PCA(n_components=8)

In [78]:
X.shape

(108, 40)

In [79]:
factor = int(np.ceil(100/29.97))

In [80]:
factor

4

In [ ]:
for frame_kp in big_list[key]:
    kp_mouth = frame_kp[0]
    x = kp_mouth[:, 0].reshape((1, -1))
    y = kp_mouth[:, 1].reshape((1, -1))
    X = np.hstack((x, y)).reshape((-1)).tolist()
    new_list.append(X)

X = np.array(new_list)

pca = PCA(n_components=8)
pca.fit(X)

## de-normalize keypoint